In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Data Preprocessing

In [3]:
#read
unsw = pd.read_csv("../datasets/UNSW_NB15_train-set.csv")

In [7]:
unsw.describe()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,175341.000000,175341.000000,175341.000000,175341.000000,1.753410e+05,1.753410e+05,1.753410e+05,175341.000000,175341.000000,1.753410e+05,...,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000
mean,87671.000000,1.359389,20.298664,18.969591,8.844844e+03,1.492892e+04,9.540619e+04,179.546997,79.609567,7.345403e+07,...,5.383538,4.206255,8.729881,0.014948,0.014948,0.133066,6.955789,9.100758,0.015752,0.680622
std,50616.731112,6.480249,136.887597,110.258271,1.747656e+05,1.436542e+05,1.654010e+05,102.940011,110.506863,1.883574e+08,...,8.047104,5.783585,10.956186,0.126048,0.126048,0.701208,8.321493,10.756952,0.124516,0.466237
min,1.000000,0.000000,1.000000,0.000000,2.800000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,43836.000000,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,3.278614e+01,62.000000,0.000000,1.305334e+04,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
50%,87671.000000,0.001582,2.000000,2.000000,4.300000e+02,1.640000e+02,3.225807e+03,254.000000,29.000000,8.796748e+05,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,1.000000
75%,131506.000000,0.668069,12.000000,10.000000,1.418000e+03,1.102000e+03,1.250000e+05,254.000000,252.000000,8.888889e+07,...,5.000000,3.000000,12.000000,0.000000,0.000000,0.000000,9.000000,12.000000,0.000000,1.000000
max,175341.000000,59.999989,9616.000000,10974.000000,1.296523e+07,1.465555e+07,1.000000e+06,255.000000,254.000000,5.988000e+09,...,51.000000,46.000000,65.000000,4.000000,4.000000,30.000000,60.000000,62.000000,1.000000,1.000000


In [8]:
#drop nulls
unsw1 = unsw.dropna()

In [9]:
y = unsw.iloc[:,-1:]

In [11]:
y.value_counts()

label
1        119341
0         56000
dtype: int64

In [13]:
119341/(119341+56000)

0.6806223302022916

In [6]:
#one hot encode protocols names
proto = pd.get_dummies(unsw['proto']) 

In [7]:
#one hot encode states
state = pd.get_dummies(unsw['state'])

In [8]:
#one hot encode service
service = pd.get_dummies(unsw['service'])
service = service.iloc[:,1:]

In [9]:
for x in ['proto', 'state', 'service','label','attack_cat']:
    unsw = unsw.drop(x,axis = 1)

In [10]:
for x in [proto, state, service]:
    # Join the encoded df
    unsw = unsw.join(x)

In [11]:
unsw

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,ssh,ssl
0,1,0.000011,2,0,496,0,90909.090200,254,0,1.803636e+08,...,0,0,0,0,0,0,0,0,0,0
1,2,0.000008,2,0,1762,0,125000.000300,254,0,8.810000e+08,...,0,0,0,0,0,0,0,0,0,0
2,3,0.000005,2,0,1068,0,200000.005100,254,0,8.544000e+08,...,0,0,0,0,0,0,0,0,0,0
3,4,0.000006,2,0,900,0,166666.660800,254,0,6.000000e+08,...,0,0,0,0,0,0,0,0,0,0
4,5,0.000010,2,0,2126,0,100000.002500,254,0,8.504000e+08,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,2,0,104,0,200000.005100,254,0,8.320000e+07,...,0,0,0,0,0,0,0,0,0,0
82328,82329,1.106101,20,8,18062,354,24.410067,254,252,1.241044e+05,...,0,0,0,0,0,0,0,0,0,0
82329,82330,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,0,0,0,0,0,0,0,0,0,0
82330,82331,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = unsw.iloc[:,:-2]

In [13]:
X

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,dhcp,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp
0,1,0.000011,2,0,496,0,90909.090200,254,0,1.803636e+08,...,0,0,0,0,0,0,0,0,0,0
1,2,0.000008,2,0,1762,0,125000.000300,254,0,8.810000e+08,...,0,0,0,0,0,0,0,0,0,0
2,3,0.000005,2,0,1068,0,200000.005100,254,0,8.544000e+08,...,0,0,0,0,0,0,0,0,0,0
3,4,0.000006,2,0,900,0,166666.660800,254,0,6.000000e+08,...,0,0,0,0,0,0,0,0,0,0
4,5,0.000010,2,0,2126,0,100000.002500,254,0,8.504000e+08,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,2,0,104,0,200000.005100,254,0,8.320000e+07,...,0,0,0,0,0,0,0,0,0,0
82328,82329,1.106101,20,8,18062,354,24.410067,254,252,1.241044e+05,...,0,0,0,0,0,0,0,0,0,0
82329,82330,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,0,0,0,0,0,0,0,0,0,0
82330,82331,0.000000,1,0,46,0,0.000000,0,0,0.000000e+00,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y

,label
0,0
1,0
2,0
3,0
4,0
...,...
82327,0
82328,0
82329,0
82330,0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
y_train

,label
66237,0
22284,1
48620,1
9317,1
21709,1
...,...
6265,1
54886,1
76820,0
860,1


# Learining Stage

## LogReg

In [61]:
clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=131, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(X_train,y_train)

/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
y_hat = clf.predict(X_test)

-------------------------
(27170, 188)
(188, 1)
-------------------------


In [63]:
y_hat

array([0, 1, 1, ..., 0, 1, 1])

In [64]:
precision_recall_fscore_support(y_test, y_hat, average='macro')

(0.7627967592011053, 0.7573183468060156, 0.746509202370998, None)

In [65]:
accuracy_score(y_test, y_hat)

0.74688995215311

In [66]:
parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              {'penalty':['none', 'elasticnet', 'l1', 'l2']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]
LogReg = LogisticRegression()
clf = GridSearchCV(LogReg, parameters)
clf.fit(X_train, y_train)

/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


-------------------------
(11033, 188)
(188, 1)
-------------------------


/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


-------------------------
(11033, 188)
(188, 1)
-------------------------


/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


...

-------------------------
(11032, 188)
(188, 1)
-------------------------


/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-------------------------
(11032, 188)
(188, 1)
-------------------------


/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/sk

GridSearchCV(estimator=LogisticRegression(),
             param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']},
                         {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100]}])

In [68]:
y_hat_grid = clf.predict(X_test)

-------------------------
(27170, 188)
(188, 1)
-------------------------


In [69]:
accuracy_score(y_test, y_hat_grid)

0.8170408538829591

In [70]:
precision_recall_fscore_support(y_test, y_hat_grid, average='macro')

(0.8180574420669884, 0.8208076983998054, 0.816780093800073, None)

In [71]:
clf.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(),
 'n_jobs': None,
 'param_grid': [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
  {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
  {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

### Gradient Boost

In [72]:
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
parameters = {
    "n_estimators":[5,50],
    "max_depth":[1,3,5,8],
    "learning_rate":[0.01,0.1,1,10,100]
}
GBC = GradientBoostingClassifier()
gbc_clf = GridSearchCV(GBC, parameters, scoring='accuracy', verbose=10)
gbc_clf.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5; 1/40] START learning_rate=0.01, max_depth=1, n_estimators=5............
[CV 1/5; 1/40] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=0.552 total time=   0.4s
[CV 2/5; 1/40] START learning_rate=0.01, max_depth=1, n_estimators=5............
[CV 2/5; 1/40] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=0.552 total time=   0.4s
[CV 3/5; 1/40] START learning_rate=0.01, max_depth=1, n_estimators=5............
[CV 3/5; 1/40] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=0.552 total time=   0.4s
[CV 4/5; 1/40] START learning_rate=0.01, max_depth=1, n_estimators=5............
[CV 4/5; 1/40] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=0.552 total time=   0.4s
[CV 5/5; 1/40] START learning_rate=0.01, max_depth=1, n_estimators=5............
[CV 5/5; 1/40] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=0.552 total time=   0.4s
[CV 1/5; 2/40] START learning_rat

[CV 1/5; 10/40] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.959 total time=   3.9s
[CV 2/5; 10/40] START learning_rate=0.1, max_depth=1, n_estimators=50...........
[CV 2/5; 10/40] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.957 total time=   4.0s
[CV 3/5; 10/40] START learning_rate=0.1, max_depth=1, n_estimators=50...........
[CV 3/5; 10/40] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.958 total time=   4.0s
[CV 4/5; 10/40] START learning_rate=0.1, max_depth=1, n_estimators=50...........
[CV 4/5; 10/40] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.957 total time=   4.0s
[CV 5/5; 10/40] START learning_rate=0.1, max_depth=1, n_estimators=50...........
[CV 5/5; 10/40] END learning_rate=0.1, max_depth=1, n_estimators=50;, score=0.961 total time=   4.0s
[CV 1/5; 11/40] START learning_rate=0.1, max_depth=3, n_estimators=5............
[CV 1/5; 11/40] END learning_rate=0.1, max_depth=3, n_estimators=5;, score=0.969 total tim

[CV 2/5; 19/40] END learning_rate=1, max_depth=3, n_estimators=5;, score=1.000 total time=   1.2s
[CV 3/5; 19/40] START learning_rate=1, max_depth=3, n_estimators=5..............
[CV 3/5; 19/40] END learning_rate=1, max_depth=3, n_estimators=5;, score=1.000 total time=   1.3s
[CV 4/5; 19/40] START learning_rate=1, max_depth=3, n_estimators=5..............
[CV 4/5; 19/40] END learning_rate=1, max_depth=3, n_estimators=5;, score=1.000 total time=   1.2s
[CV 5/5; 19/40] START learning_rate=1, max_depth=3, n_estimators=5..............
[CV 5/5; 19/40] END learning_rate=1, max_depth=3, n_estimators=5;, score=1.000 total time=   1.2s
[CV 1/5; 20/40] START learning_rate=1, max_depth=3, n_estimators=50.............
[CV 1/5; 20/40] END learning_rate=1, max_depth=3, n_estimators=50;, score=1.000 total time=   4.0s
[CV 2/5; 20/40] START learning_rate=1, max_depth=3, n_estimators=50.............
[CV 2/5; 20/40] END learning_rate=1, max_depth=3, n_estimators=50;, score=1.000 total time=   4.1s
[CV 3

[CV 3/5; 28/40] END learning_rate=10, max_depth=3, n_estimators=50;, score=0.597 total time=   9.9s
[CV 4/5; 28/40] START learning_rate=10, max_depth=3, n_estimators=50............
[CV 4/5; 28/40] END learning_rate=10, max_depth=3, n_estimators=50;, score=0.045 total time=   9.3s
[CV 5/5; 28/40] START learning_rate=10, max_depth=3, n_estimators=50............
[CV 5/5; 28/40] END learning_rate=10, max_depth=3, n_estimators=50;, score=0.052 total time=  10.0s
[CV 1/5; 29/40] START learning_rate=10, max_depth=5, n_estimators=5.............
[CV 1/5; 29/40] END learning_rate=10, max_depth=5, n_estimators=5;, score=0.763 total time=   1.1s
[CV 2/5; 29/40] START learning_rate=10, max_depth=5, n_estimators=5.............
[CV 2/5; 29/40] END learning_rate=10, max_depth=5, n_estimators=5;, score=0.622 total time=   1.7s
[CV 3/5; 29/40] START learning_rate=10, max_depth=5, n_estimators=5.............
[CV 3/5; 29/40] END learning_rate=10, max_depth=5, n_estimators=5;, score=0.552 total time=   1.4

[CV 4/5; 37/40] END learning_rate=100, max_depth=5, n_estimators=5;, score=0.643 total time=   1.8s
[CV 5/5; 37/40] START learning_rate=100, max_depth=5, n_estimators=5............
[CV 5/5; 37/40] END learning_rate=100, max_depth=5, n_estimators=5;, score=0.714 total time=   1.4s
[CV 1/5; 38/40] START learning_rate=100, max_depth=5, n_estimators=50...........
[CV 1/5; 38/40] END learning_rate=100, max_depth=5, n_estimators=50;, score=0.439 total time=  16.0s
[CV 2/5; 38/40] START learning_rate=100, max_depth=5, n_estimators=50...........
[CV 2/5; 38/40] END learning_rate=100, max_depth=5, n_estimators=50;, score=0.245 total time=  14.2s
[CV 3/5; 38/40] START learning_rate=100, max_depth=5, n_estimators=50...........
[CV 3/5; 38/40] END learning_rate=100, max_depth=5, n_estimators=50;, score=0.249 total time=  14.5s
[CV 4/5; 38/40] START learning_rate=100, max_depth=5, n_estimators=50...........
[CV 4/5; 38/40] END learning_rate=100, max_depth=5, n_estimators=50;, score=0.643 total time

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 1, 10, 100],
                         'max_depth': [1, 3, 5, 8], 'n_estimators': [5, 50]},
             scoring='accuracy', verbose=10)

In [85]:
y_hat_gbc_grid = gbc_clf.predict(X_test)

In [87]:
accuracy_score(y_test, y_hat_gbc_grid)

1.0

In [89]:
precision_recall_fscore_support(y_test, y_hat_gbc_grid, average='macro')

(1.0, 1.0, 1.0, None)